In [15]:
import os
import requests
from bs4 import BeautifulSoup
import textwrap
import pandas as pd
import lxml  
import time
from IPython.display import clear_output
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from time import sleep
import re
from tqdm import tqdm
import psycopg2
from sqlalchemy import create_engine

In [6]:
class keywords(object):
    smartphone = 'smartphone'
    notebook = 'notebook'

In [ ]:
# Using selenium app from Docker

selenuim_driver = webdriver.Remote("http://selenium:4444/wd/hub", DesiredCapabilities.CHROME)

# Using chromedriver and Chrome browser installed somewhere (when running outside Docker container)

# chromedriver_path = '_some_path_/chromedriver.exe'
# selenuim_driver = webdriver.Chrome(chromedriver_path, options = Options())

In [12]:
class Driver(object):
    def __init__(self, driver):
        self.driver = driver

    def getLink(self, link_text, waiting_time = 1, niters = 5, raise_error = True):
        for _ in range(niters):
            try:
                return self.driver.find_element_by_partial_link_text(link_text)
            except:
                sleep(waiting_time)
        
        if raise_error:
            raise(Exception("Driver: Failed to get link"))
        
    def getHTML(self, url, waiting_time = 2):
        self.driver.get(url)
        
        sleep(waiting_time)

        content = self.driver.page_source

        return BeautifulSoup(content, 'html.parser')
    
    def processRequest(self, url, soup_function, waiting_time = 1, niters = 1, raise_error = True, load_url = True):
        if load_url:
            self.driver.get(url)
            sleep(waiting_time)
        
        for _ in range(niters):
            try:
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                return soup_function(soup)
            except:
                sleep(waiting_time)
        
        if raise_error:
            raise(Exception("Driver: Failed to process request"))
        
driver = Driver(selenuim_driver)

In [13]:
class DNS_shop_parser(object):
    def __init__(self, driver):
        self.driver = driver
        self.root_url = "https://www.dns-shop.ru"
        self.default_city_en = "Moscow"
        self.default_city_ru = "Москва"
        
        self.product_urls = {
                keywords.smartphone: "/".join((self.root_url, 'catalog/17a8a01d16404e77/smartfony')),
                keywords.notebook: "/".join((self.root_url, 'catalog/17a892f816404e77/noutbuki'))
            }
        
        self.link_keywords = {
                keywords.smartphone: ["Смартфон "],
                keywords.notebook: ["Ноутбук ", "Ультрабук ", "Нетбук "]
            }
        
        self.link_patterns = {
                keywords.smartphone: " .*?фон ",
                keywords.notebook: " .*?бук "
            }
        
        self.setDefaultLocation()
        
    def setDefaultLocation(self):
        soup = self.parseURL('/'.join([self.root_url, f'?city={self.default_city_en}']))

        current_city = None
        try:
            current_city = soup.find("div", {"class": "w-choose-city-widget-label"}).getText()
        except:
            raise(Exception("DNS_shop_parser: Failed to set default location"))

        if current_city == self.default_city_ru:
            print('default location is set')
        else:
            raise(Exception("DNS_shop_parser: Failed to set default location"))
        
    def parseURL(self, url, waiting_time = 2):
        return self.driver.getHTML(url, waiting_time)
    
    def getPageLinks(self, product_keyword, page_num = 1):
        url = self.product_urls[product_keyword]
        pattern = self.link_patterns[product_keyword]
        
        soup = self.parseURL("/".join((url, f'?p={page_num}')))
        
        return soup.find_all("a", string = re.compile(pattern))
    
    def parseProductName(self, name, product_keyword):
        keywords = self.link_keywords[product_keyword]
        
        for keyword in keywords:
            if re.search(keyword, name) is not None:
                return name[name.find(keyword) + len(keyword):]
        
    def parseRawLinks(self, raw_links, product_keyword):
        hrefs = [self.root_url + link.get('href') for link in raw_links]
        names = [link.string for link in raw_links]
        names = list(map(lambda name: self.parseProductName(name, product_keyword), names))
        
        return [{'name': name, 'href': href} for (name, href) in zip(names, hrefs)]
    
    def getAllLinks(self, product_keyword, npages = 100):
        page_num = 1
        
        all_links = []
        
        for page_num in tqdm(range(1, npages + 1)):          
            links = self.getPageLinks(product_keyword, page_num)
            
            if len(links) == 0:
                break
            
            all_links.extend(links)
            sleep(5)
            
            if page_num % 5 == 0:
                pd.DataFrame(self.parseRawLinks(all_links, product_keyword)).\
                        to_csv(f'{product_keyword}_pages.csv', encoding = 'utf-16')

        return self.parseRawLinks(all_links, product_keyword)   

parser = DNS_shop_parser(driver)

default location is set


In [29]:
smartphones = parser.getAllLinks(keywords.smartphone, npages = 100)
smartphones[:5], len(smartphones)

100%|██████████| 1/1 [00:11<00:00, 11.10s/it]


([{'name': 'DEXP G450 8 ГБ серый',
   'href': 'https://www.dns-shop.ru/product/86c5d33d07813332/5-smartfon-dexp-g450-8-gb-seryj/'},
  {'name': 'DEXP G450 8 ГБ синий',
   'href': 'https://www.dns-shop.ru/product/d4aa88fc0bf43332/5-smartfon-dexp-g450-8-gb-sinij/'},
  {'name': 'Nobby S300 Pro 16 ГБ золотистый',
   'href': 'https://www.dns-shop.ru/product/8560f9e6c80f3332/495-smartfon-nobby-s300-pro-16-gb-zolotistyj/'},
  {'name': 'Nobby S300 Pro 16 ГБ серый',
   'href': 'https://www.dns-shop.ru/product/e833735fc80f3332/495-smartfon-nobby-s300-pro-16-gb-seryj/'},
  {'name': 'Nobby S300 Pro 16 ГБ синий',
   'href': 'https://www.dns-shop.ru/product/3c314766c8103332/495-smartfon-nobby-s300-pro-16-gb-sinij/'}],
 18)

In [95]:
notebooks = parser.getAllLinks(keywords.notebook, npages = 100)
notebooks[:5], len(notebooks)

 41%|█████████████████████████████████▏                                               | 41/100 [08:05<11:38, 11.83s/it]


([{'name': 'Digma EVE 10 C301 черный',
   'href': 'https://www.dns-shop.ru/product/a716b761336f1b80/101-netbuk-digma-eve-10-c301-cernyj/'},
  {'name': 'ASUS Laptop E210MA-GJ001T синий',
   'href': 'https://www.dns-shop.ru/product/5c01fb1e84773332/116-noutbuk-asus-laptop-e210ma-gj001t-sinij/'},
  {'name': 'ASUS Laptop E210MA-GJ002T розовый',
   'href': 'https://www.dns-shop.ru/product/62131bcd84773332/116-noutbuk-asus-laptop-e210ma-gj002t-rozovyj/'},
  {'name': 'ASUS Laptop E210MA-GJ003T белый',
   'href': 'https://www.dns-shop.ru/product/62131bce84773332/116-noutbuk-asus-laptop-e210ma-gj003t-belyj/'},
  {'name': 'ASUS Laptop K540BA-GQ613 черный',
   'href': 'https://www.dns-shop.ru/product/af32176fcd753332/156-noutbuk-asus-laptop-k540ba-gq613-cernyj/'}],
 729)

In [42]:
class DNS_product_parser(object):    
    def __init__(self, driver):
        self.driver = driver
        self.root_url = "https://www.dns-shop.ru"
        self.default_city_en = "Moscow"
        self.default_city_ru = "Москва"

        self.topics_parsers = {
            'description': self.getDescription, 
            'characteristics': self.getCharacteristics, 
            'opinion': self.getOpinions, 
            'comment': self.getComments, 
            'price': self.getPrice,
            'rating': self.getRating,
        }
        
        self.setDefaultLocation()
        
    def setDefaultLocation(self):
        soup = self.parseURL('/'.join([self.root_url, f'?city={self.default_city_en}']))

        current_city = None
        try:
            current_city = soup.find("div", {"class": "w-choose-city-widget-label"}).getText()
        except:
            raise(Exception("DNS_product_parser: Failed to set default location"))

        if current_city == self.default_city_ru:
            print('default location is set')
        else:
            raise(Exception("DNS_product_parser: Failed to set default location"))
            
    def parseURL(self, url, waiting_time = 2):
        return self.driver.getHTML(url, waiting_time)
    
    def getPrice(self, link):
        url = link['href']

        soup_function = lambda soup: float(soup.find("span", {"class": re.compile(r'^product-card-price__current')})\
                                              .getText()[:-2].replace(' ', ''))
        price = self.driver.processRequest(url, soup_function, niters = 100, load_url = False)

        return {'price': price}
    
    def getRating(self, link):
        url = link['href']

        soup_function = lambda soup: float(soup.find("span", {"itemprop": "ratingValue"}).getText())
        average_rating = self.driver.processRequest(url, soup_function, niters = 3, raise_error = False, load_url = False)
        
        soup_function = lambda soup: int(soup.find("span", {"itemprop": "ratingCount"}).getText())
        reviews_num = self.driver.processRequest(url, soup_function, niters = 3, raise_error = False, load_url = False)
        
        return {'average_rating': average_rating, 'reviews_num': reviews_num}

    
    def getDescription(self, link):
        url = link['href'] + 'description'
        soup = self.parseURL(url)
        description_string = soup.find("div", {"class": "price-item-description"}).find('p').getText()

        return {'description': description_string}
    
    def getCharacteristics(self, link):
        url = link['href'] + 'characteristics'
        
        soup = self.parseURL(url)
        
        characteristics_table = soup.find('table')
        df = pd.read_html(str(characteristics_table))[0]
        df = df.drop(df[df[0] == df[1]].index)
        chars_dict = dict(zip(df[0],df[1]))

        return chars_dict

    def getOpinions(self, link):
        url = link['href'] + 'opinion'

        soup = self.parseURL(url)
        
        ratings_num = {}

        for i in range(1, 6):
            ratings_num["rating " + str(i)] = int(soup.find("div", {"class": "ow-counts"}).\
                                                  find("a", {"data-rating": str(i)}).getText().split()[0])
    
    def getComments(self, link):
        pass
    
    def parseTopics(self, links, topics = ['characteristics', 'rating', 'price']):
        links_values = pd.DataFrame()
        
        for link in tqdm(links):
            link_vals = {}
            
            brand_name, model_name = link['name'][: link['name'].find(' ')], link['name'][link['name'].find(' ') + 1:]

            for topic in topics:
                topic_parser = self.topics_parsers[topic]
                link_vals.update(topic_parser(link))
                
            clear_output(wait = True)
                
            links_values = links_values.append(link_vals, ignore_index = True)
            
            links_values.to_csv('notebooks.csv', encoding = 'utf-16')
            
        return links_values

product_parser = DNS_product_parser(driver)

default location is set


In [102]:
links_values = product_parser.parseTopics(notebooks)

100%|██████████████████████████████████████████████████████████████████████████████| 729/729 [1:59:54<00:00,  9.87s/it]


In [36]:
links_values

,CrossFire/SLI-массив,SSHD накопитель (объем SSD буфера),average_rating,price,reviews_num,Автоматическое увеличение частоты,Архитектура процессора,Аудио интерфейсы,Беспроводные виды доступа в Интернет,Веб-камера,...,Цветовой охват,Цифровой блок клавиатуры,Частота,Частота оперативной памяти,Ширина,Год релиза,Количество слотов под модули памяти,"Особенности, дополнительно",Скорость сетевого адаптера,Технология динамического обновления экрана
0,нет,нет,1.0,17999.0,1.0,2.2 ГГц,GoldMont,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,"49% sRGB, 45% NTSC, 34% AdobeRGB",нет,1.1 ГГц,2133 МГц,268 мм,NaN,NaN,NaN,NaN,NaN
1,нет,нет,4.2,24999.0,17.0,2.8 ГГц,Goldmont Plus,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,45% NTSC,есть,1.1 ГГц,2400 МГц,279 мм,2020.0,интегрирована,NumberPad,нет,нет
2,нет,нет,4.7,24999.0,3.0,2.8 ГГц,Goldmont Plus,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,45% NTSC,есть,1.1 ГГц,2400 МГц,279 мм,2020.0,интегрирована,NumberPad,нет,нет
3,нет,нет,4.6,24999.0,9.0,2.8 ГГц,Goldmont Plus,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,45% NTSC,нет,1.1 ГГц,2133 МГц,279 мм,2020.0,интегрирована,NumberPad,нет,нет
4,нет,нет,3.8,28999.0,9.0,2.6 ГГц,Excavator,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,NaN,есть,2.3 ГГц,1866 МГц,381 мм,2020.0,1,NaN,нет,нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,нет,нет,NaN,313999.0,NaN,4.5 ГГц,Comet Lake,"3.5 мм jack (наушники), 3.5 мм jack (микрофонный)",Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2666 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет
725,нет,нет,3.0,349999.0,2.0,4.5 ГГц,Comet Lake,"3.5 мм jack (наушники), 3.5 мм jack (микрофонный)",Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2666 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет
726,нет,нет,NaN,379999.0,NaN,5 ГГц,Comet Lake,3.5 мм jack (микрофон/аудио),Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2933 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет
727,нет,нет,NaN,380999.0,NaN,4.5 ГГц,Comet Lake,"3.5 мм jack (наушники), 3.5 мм jack (микрофонный)",Wi-Fi,есть,...,100% AdobeRGB,нет,2.6 ГГц,2666 МГц,358.5 мм,2020.0,2,NaN,1000 Мбит,нет


In [2]:
notebooks = pd.read_csv('notebook_pages.csv', encoding = 'utf-16', index_col = 0)
notebooks

,name,href
0,Digma EVE 10 C301 черный,https://www.dns-shop.ru/product/a716b761336f1b...
1,ASUS Laptop E210MA-GJ001T синий,https://www.dns-shop.ru/product/5c01fb1e847733...
2,ASUS Laptop E210MA-GJ002T розовый,https://www.dns-shop.ru/product/62131bcd847733...
3,ASUS Laptop E210MA-GJ003T белый,https://www.dns-shop.ru/product/62131bce847733...
4,ASUS Laptop K540BA-GQ613 черный,https://www.dns-shop.ru/product/af32176fcd7533...
...,...,...
715,Apple MacBook Pro Retina TB (MVVK2RU/A) серый,https://www.dns-shop.ru/product/c2682529067733...
716,Apple MacBook Pro Retina TB (MVVM2RU/A) серебр...,https://www.dns-shop.ru/product/6dd67002067833...
717,Acer ConceptD 7 Pro CN715-71P-79QK белый,https://www.dns-shop.ru/product/61ff6a29d60333...
718,Acer Predator Helios 700 PH717-72-94AW черный,https://www.dns-shop.ru/product/4f947950dd2433...


In [45]:
product_parser.getCharacteristics(notebooks.iloc[0])

{'Гарантия': '12 мес.',
 'Страна-производитель': 'Китай',
 'Тип': 'нетбук',
 'Операционная система': 'Windows 10 Home',
 'Модель': 'Digma EVE 10 C301',
 'Код производителя': 'ES1050EW',
 'Год релиза': '2020',
 'Игровой ноутбук': 'нет',
 'Цвет верхней крышки': 'черный',
 'Материал корпуса': 'пластик',
 'Конструктивное исполнение': 'классический',
 'Цифровой блок клавиатуры': 'нет',
 'Подсветка клавиш': 'нет',
 'Тип экрана': 'IPS',
 'Диагональ экрана (дюйм)': '10.1"',
 'Разрешение экрана': '1280x800',
 'Название формата': 'HD',
 'Время отклика пикселя': '30 мс',
 'Плотность пикселей': '149 ppi',
 'Максимальная частота обновления экрана': '60 Гц',
 'Цветовой охват': '49% sRGB, 34% AdobeRGB, 45% NTSC',
 'Покрытие экрана': 'матовое',
 'Сенсорный экран': 'нет',
 'Производитель процессора': 'Intel',
 'Линейка процессора': 'Celeron',
 'Модель процессора': 'N3450',
 'Количество ядер процессора': '4',
 'Максимальное число потоков': '4',
 'Частота': '1.1 ГГц',
 'Автоматическое увеличение частоты'

In [27]:
conn_string = "host = 'db' dbname = 'postgres' user = 'postgres' password = 'admin'"
conn = psycopg2.connect(conn_string)

In [31]:
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS notebook_models')
cursor.execute('COMMIT')

In [32]:
engine = create_engine('postgresql://postgres:admin@db:5432/postgres')

In [33]:
notebooks.to_sql('notebook_models', engine)

In [34]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM notebook_models LIMIT 2')
cursor.fetchall() 

[(0,
  'Digma EVE 10 C301 черный',
  'https://www.dns-shop.ru/product/a716b761336f1b80/101-netbuk-digma-eve-10-c301-cernyj/'),
 (1,
  'ASUS Laptop E210MA-GJ001T синий',
  'https://www.dns-shop.ru/product/5c01fb1e84773332/116-noutbuk-asus-laptop-e210ma-gj001t-sinij/')]

In [35]:
pd.read_sql_query("SELECT * FROM notebook_models", con = engine)

,index,name,href
0,0,Digma EVE 10 C301 черный,https://www.dns-shop.ru/product/a716b761336f1b...
1,1,ASUS Laptop E210MA-GJ001T синий,https://www.dns-shop.ru/product/5c01fb1e847733...
2,2,ASUS Laptop E210MA-GJ002T розовый,https://www.dns-shop.ru/product/62131bcd847733...
3,3,ASUS Laptop E210MA-GJ003T белый,https://www.dns-shop.ru/product/62131bce847733...
4,4,ASUS Laptop K540BA-GQ613 черный,https://www.dns-shop.ru/product/af32176fcd7533...
...,...,...,...
715,715,Apple MacBook Pro Retina TB (MVVK2RU/A) серый,https://www.dns-shop.ru/product/c2682529067733...
716,716,Apple MacBook Pro Retina TB (MVVM2RU/A) серебр...,https://www.dns-shop.ru/product/6dd67002067833...
717,717,Acer ConceptD 7 Pro CN715-71P-79QK белый,https://www.dns-shop.ru/product/61ff6a29d60333...
718,718,Acer Predator Helios 700 PH717-72-94AW черный,https://www.dns-shop.ru/product/4f947950dd2433...
